In [ ]:
from IPython.display import clear_output
import os
import time
import shutil

!apt-get update
!apt-get install libgl1-mesa-glx -y
%pip install --upgrade pip setuptools

%pip install -r requirements.txt
    
%pip install torch==2.2.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --upgrade
%pip install triton
%pip install xformers==0.0.24
%pip install diffusers==0.26.3
   
# clear_output()
print("dependencies installed.")

# disk info
def get_storage_stats():
    # Run the df command and capture its output
    output = !df -h /
    disk_info = output[1].split()    
    total_storage = disk_info[1]
    used_storage = disk_info[2]    
    return f"Storage: {used_storage} used out of {total_storage}"




In [2]:
#Make model dirs
# import os
# import shutil

V0F_PATH = "models/SUPIR-v0F"
V0Q_PATH = "models/SUPIR-v0Q"
SDXL_BASE_MODEL_PATH = "models/SDXL_Base"

SDXL_CLIP1_PATH = "models/clip-vit-large-patch14"
SDXL_CLIP2_CKPT_PTH = "models/CLIP-ViT-bigG-14-laion2B-39B-b160k"
# LLAVA_CLIP_PATH = "models/clip-vit-large-patch14-336"
# LLAVA_MODEL_PATH = "models/llava-v1.5-13b"

# Ensure directories exist
os.makedirs(V0F_PATH, exist_ok=True)
os.makedirs(V0Q_PATH, exist_ok=True)
os.makedirs(SDXL_CLIP1_PATH, exist_ok=True)
os.makedirs(SDXL_CLIP2_CKPT_PTH, exist_ok=True)
os.makedirs(SDXL_BASE_MODEL_PATH, exist_ok=True)
# os.makedirs(LLAVA_CLIP_PATH, exist_ok=True)
# os.makedirs(LLAVA_MODEL_PATH, exist_ok=True)

print("Finished creating directories.")

In [ ]:
# Create a downloads directory
downloads_dir = "downloads"
os.makedirs(downloads_dir, exist_ok=True)


# URLs and their respective extraction paths
models_info = [
    ('https://huggingface.co/yushan777/SUPIR/resolve/main/SUPIR-v0F.ckpt', V0F_PATH),
    ('https://huggingface.co/yushan777/SUPIR/resolve/main/SUPIR-v0Q.ckpt', V0Q_PATH),
    ('https://huggingface.co/yushan777/SUPIR/resolve/main/sd_xl_base_1.0_0.9vae.safetensors',SDXL_BASE_MODEL_PATH),
    ('https://huggingface.co/yushan777/SUPIR/resolve/main/clip-vit-large-patch14.tar', SDXL_CLIP1_PATH),
    ('https://huggingface.co/yushan777/SUPIR/resolve/main/CLIP-ViT-bigG-14-laion2B-39B-b160k.tar', SDXL_CLIP2_CKPT_PTH)
]

print("This will take a while...\n")

# Download and extract loop
for url, extraction_path in models_info:
    # Extract the filename from the URL
    filename = url.split('/')[-1]
    download_path = os.path.join(downloads_dir, filename)
    
    # Download the file (minimal output + progress)
    print(f"Downloading {filename} from {url} to {download_path}...")
    !wget -q --show-progress {url} -O {download_path}  # Download into the downloads directory
    
    # Check if the file is a .tar file for extraction
    if filename.endswith('.tar'):
        print(f"Extracting {filename} to {extraction_path}...")
        !tar -xf {download_path} -C {extraction_path}
    else:
        # Since it's not a .tar file, copy it directly to the extraction path
        print(f"Copying {filename} to {extraction_path}")
        destination_path = os.path.join(extraction_path, filename)
        shutil.copy(download_path, destination_path)
    
    # Pause for 3 seconds
    # can help mitigate the rare occasional corruption when extracting large files 
    time.sleep(3)  

    # Optionally, remove the comment from the next line if you wish to delete the tar file after extraction
    # !rm {download_path}

print("Finished downloading models.")

In [ ]:

# float16() - requires under 12GB VRAM
python3 gradio_demo.py --share --no_llava --loading_half_params --use_tile_vae --use_image_slider 